## Creating a table with PostgreSQL

In [1]:
import psycopg2
import pandas as pd

#### Created a connection to the database and used the connection to get a cursor to execute queries and also created a database accounts.

In [2]:
def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # creating database 
    
    cur.execute('DROP DATABASE IF EXISTS accounts')
    cur.execute('CREATE DATABASE accounts')
    
    # close the current connection
    
    conn.close()
    
    # connect to accounts database
    
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
CustomerData = pd.read_csv("customer_data.csv")

In [4]:
CustomerData.head()

,customer_id,gender,age,payment_method
0,C241288,Female,28.0,Credit Card
1,C111565,Male,21.0,Debit Card
2,C266599,Male,20.0,Cash
3,C988172,Female,66.0,Credit Card
4,C189076,Female,53.0,Cash


In [5]:
SalesData = pd.read_csv("sales_data.csv")

In [6]:
SalesData.columns

Index(['invoice_no', 'customer_id', 'category', 'quantity', 'price',
       'invoice_date', 'shopping_mall'],
      dtype='object')

In [7]:
SalesData.head()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall
0,I138884,C241288,Clothing,5,1500.40,05-08-2022,Kanyon
1,I317333,C111565,Shoes,3,1800.51,12-12-2021,Forum Istanbul
2,I127801,C266599,Clothing,1,300.08,09-11-2021,Metrocity
3,I173702,C988172,Shoes,5,3000.85,16-05-2021,Metropol AVM
4,I337046,C189076,Books,4,60.60,24-10-2021,Kanyon


In [8]:
CustomerData.head()

,customer_id,gender,age,payment_method
0,C241288,Female,28.0,Credit Card
1,C111565,Male,21.0,Debit Card
2,C266599,Male,20.0,Cash
3,C988172,Female,66.0,Credit Card
4,C189076,Female,53.0,Cash


In [9]:
cur,conn = create_database()

#### Create a table for customerdata 

In [10]:
customerdata_table_create = ("""CREATE TABLE IF NOT EXISTS customerdata(
customer_id VARCHAR PRIMARY KEY,
gender VARCHAR,
age NUMERIC,
payment_method VARCHAR)
""")

cur.execute(customerdata_table_create)
conn.commit()

In [11]:
SalesData.head()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall
0,I138884,C241288,Clothing,5,1500.40,05-08-2022,Kanyon
1,I317333,C111565,Shoes,3,1800.51,12-12-2021,Forum Istanbul
2,I127801,C266599,Clothing,1,300.08,09-11-2021,Metrocity
3,I173702,C988172,Shoes,5,3000.85,16-05-2021,Metropol AVM
4,I337046,C189076,Books,4,60.60,24-10-2021,Kanyon


In [12]:
SalesData.drop(columns=['invoice_date'],inplace=True)

#### Create a table for salesdata

In [13]:
Salesdata_table_create = ("""CREATE TABLE IF NOT EXISTS salesdata(
invoice_no VARCHAR,
customer_id VARCHAR,
category VARCHAR,
quantity NUMERIC,
price NUMERIC,
shopping_mall VARCHAR)
""")

cur.execute(Salesdata_table_create)
conn.commit()

#### Inserting the rows into the customerdata table

In [14]:
customer_data_table_insert = """INSERT INTO customerdata(customer_id,gender,age,payment_method) VALUES (%s,%s,%s,%s)"""

In [15]:
for i,row in CustomerData.iterrows():
    cur.execute(customer_data_table_insert, list(row))
    #print(row)

#### Inserting the rows into the salesdata table

In [16]:
sales_data_table_insert = ("""INSERT INTO salesdata(invoice_no,customer_id,category,quantity,price,shopping_mall)VALUES (%s,%s,%s,%s,%s,%s)""")

In [17]:
c = len(CustomerData)
print(c)

99457


In [18]:
d = len(SalesData)
print(d)

99457


In [19]:
for i,row in SalesData.iterrows():
    cur.execute(sales_data_table_insert, list(row))
    #print(i)

In [20]:
cur.execute('select * from customerdata limit 5')

### Validate your data was inserted into the respective tables

In [21]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('C241288', 'Female', Decimal('28.0'), 'Credit Card')
('C111565', 'Male', Decimal('21.0'), 'Debit Card')
('C266599', 'Male', Decimal('20.0'), 'Cash')
('C988172', 'Female', Decimal('66.0'), 'Credit Card')
('C189076', 'Female', Decimal('53.0'), 'Cash')


In [22]:
cur.execute('select * from salesdata limit 5')

In [23]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('I138884', 'C241288', 'Clothing', Decimal('5'), Decimal('1500.4'), 'Kanyon')
('I317333', 'C111565', 'Shoes', Decimal('3'), Decimal('1800.51'), 'Forum Istanbul')
('I127801', 'C266599', 'Clothing', Decimal('1'), Decimal('300.08'), 'Metrocity')
('I173702', 'C988172', 'Shoes', Decimal('5'), Decimal('3000.85'), 'Metropol AVM')
('I337046', 'C189076', 'Books', Decimal('4'), Decimal('60.6'), 'Kanyon')


### Finally close your cursor and connection

In [24]:
cur.close()
conn.close()